# MLFlow tracking Server

* You can use MLFlow to compare multiple experiments, metrics and parameters.

Use ```
mlflow ui``` in the command line to open the mlflow server that keeps track of all the experiments



In [13]:
import mlflow
import pandas as pd
from mlflow.models import infer_signature 
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

## Data prep and model training

In [3]:
# Load dataset
X,y = datasets.load_iris(return_X_y = True)

# Split teh data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2)

# Define model hyperparameters
params = {"penalty":"l2","solver":"lbfgs","max_iter":1000,"multi_class":"auto", "random_state":8888}

# Train the model
lr = LogisticRegression(**params)
lr.fit(X_train,y_train)

c:\Users\Sebastian Leon\Documents\VirtualEnvironments\mlbootcamp_venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,8888
,solver,'lbfgs'
,max_iter,1000
,multi_class,'auto'


In [4]:
# Predict on the test set
y_pred = lr.predict(X_test)

In [5]:
# Calculate accuracy

accuracy = accuracy_score(y_test,y_pred)
print(accuracy)

0.9333333333333333


## MLFlow tracking

In [14]:
# Any experiment will be tracked in that URL. Use HTTP, no HTTPS
mlflow.set_tracking_uri("http://127.0.0.1:5000")

# Create a new mlflow experiment
mlflow.set_experiment("MLFLOW Quickstart")

# To start the url tracking use mlflow ui

<Experiment: artifact_location='mlflow-artifacts:/360330419078114691', creation_time=1756164283833, experiment_id='360330419078114691', last_update_time=1756164283833, lifecycle_stage='active', name='MLFLOW Quickstart', tags={}>

In [ ]:
# Track parameters + start MLFLOW run
with mlflow.start_run():
    # Log hyperparameters
    mlflow.log_params(params)

    # Log accuracy metrics
    mlflow.log_metric("accuracy",accuracy) # They must be key value pairs

    # Set tag
    mlflow.set_tag("Training info","Basic LR model for isis data")

    # Infer the model signature
    signature = infer_signature(X_train,lr.predict(X_train))

    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=lr,
        name="iris_model", # artifact_path deprecated
        signature=signature,
        input_example=X_train,
        registered_model_name="training-quickstart"
    )

2025/08/25 18:25:35 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
Successfully registered model 'training-quickstart'.
2025/08/25 18:25:50 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: training-quickstart, version 1
Created version '1' of model 'training-quickstart'.


🏃 View run wise-seal-617 at: http://127.0.0.1:5000/#/experiments/360330419078114691/runs/19374488734a49fc8456a11d32dacee3
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/360330419078114691


## Create a new model with other parameters

In [15]:
# Define model hyperparameters
params = {"solver":"newton-cg","max_iter":1000,"multi_class":"auto", "random_state":1000}

# Train the model
lr = LogisticRegression(**params)
lr.fit(X_train,y_train)

c:\Users\Sebastian Leon\Documents\VirtualEnvironments\mlbootcamp_venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,1000
,solver,'newton-cg'
,max_iter,1000
,multi_class,'auto'


In [16]:
# Predict on the test set
y_pred = lr.predict(X_test)

In [17]:
# Calculate accuracy
accuracy = accuracy_score(y_test,y_pred)
print(accuracy)

0.9333333333333333


In [18]:
# Track parameters + start MLFLOW run
with mlflow.start_run():
    # Log hyperparameters
    mlflow.log_params(params)

    # Log accuracy metrics
    mlflow.log_metric("accuracy",accuracy) # They must be key value pairs

    # Set tag
    mlflow.set_tag("Training info","Basic LR model for isis data")

    # Infer the model signature
    signature = infer_signature(X_train,lr.predict(X_train))

    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=lr,
        name="iris_model", # artifact_path deprecated
        signature=signature,
        input_example=X_train,
        registered_model_name="training-quickstart"
    )

Registered model 'training-quickstart' already exists. Creating a new version of this model...
2025/08/25 19:35:36 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: training-quickstart, version 3
Created version '3' of model 'training-quickstart'.


🏃 View run inquisitive-eel-219 at: http://127.0.0.1:5000/#/experiments/360330419078114691/runs/9893f881ff5c44208ed25657de447b96
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/360330419078114691


## Load the model back for prediction as a generic python function

In [21]:
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)

predictions = loaded_model.predict(X_test)

iris_features_names = datasets.load_iris().feature_names
result = pd.DataFrame(X_test,columns = iris_features_names)

result["actual_class"] = y_test
result["predicted"] = predictions
result.head(2)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),actual_class,predicted
0,7.3,2.9,6.3,1.8,2,2
1,7.7,3.0,6.1,2.3,2,2


## Model registry

Centralized store, set of APIs and UI to collaborately manage the full lifecycle of an MLFlow model. It provides model lineage, versioning, aliasing, tagging and annotations.

Do not register models at the beggining, only after it is the best model

In [22]:
# Track parameters + start MLFLOW run
with mlflow.start_run():
    # Log hyperparameters
    mlflow.log_params(params)

    # Log accuracy metrics
    mlflow.log_metric("accuracy",0.97) # They must be key value pairs

    # Set tag
    mlflow.set_tag("Training info","Basic LR model for isis data")

    # Infer the model signature
    signature = infer_signature(X_train,lr.predict(X_train))

    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=lr,
        name="iris_model", # artifact_path deprecated
        signature=signature,
        input_example=X_train
    )

🏃 View run respected-elk-298 at: http://127.0.0.1:5000/#/experiments/360330419078114691/runs/e918ee97a3ae47768d29c6bd305bc9d6
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/360330419078114691


## Inferencing from model registry

In [26]:
import mlflow.sklearn

model_name = "training-quickstart"
model_version = "latest"
model_uri = f"models:/{model_name}/{model_version}"

model = mlflow.sklearn.load_model(model_uri)
model

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,1000
,solver,'newton-cg'
,max_iter,1000
,multi_class,'auto'


In [27]:
y_pred_new = model.predict(X_test)
y_pred_new

array([2, 2, 0, 2, 0, 2, 2, 0, 0, 2, 1, 2, 1, 1, 1, 0, 0, 2, 2, 2, 1, 0,
       0, 0, 1, 1, 1, 1, 1, 2])